**Сначала на основе обучающей выборки строятся эмбеддинги, используя fasttext. Для каждого названия организации строится один вектор, равный сумме эмбеддингов слов, из которых состоит название. Каждому роду занятий ставится в соответствие номер, который и должна предсказывать модель. Т.е. получаем задачу, где вектор признаков объекта - его эмбеддинг, а метка класса - номер рода деятельности. Названия родов деятельности добавлены о обучающую выборку. По обучающей выборке обучаем knn c k=5, полученная модель делает предсказание.**

In [1]:
file = open('names_and_rubrics_learn.tsv', 'r')
train = [line.strip().split('\t') for line in file]
file.close()

In [2]:
import fasttext

In [3]:
#model = fasttext.skipgram('names_and_rubrics_learn.tsv', 'model')
model = fasttext.load_model('model.bin')

In [4]:
file2 = open('names_and_rubrics_test_no_rubric.tsv', 'r')
test = [line.strip() for line in file2]
file2.close()

In [7]:
import numpy as np

In [8]:
from collections import defaultdict
rubric_id = defaultdict()
id_rubric = defaultdict()
id_ = 0
for i in range(len(train)):
    if train[i][1] not in rubric_id:
        rubric_id[train[i][1]] = id_
        id_rubric[id_] = train[i][1]
        id_ += 1

In [9]:
train_X = np.zeros((len(train) + len(rubric_id), len(model['hello'])))

In [10]:
for i in range(len(train)):
    x_words = train[i][0].split(' ')
    for x_w in x_words:
        train_X[i, :] += np.array(model[x_w])
for i in range(len(id_rubric)):
    x_words = id_rubric[i].split(' ')
    for x_w in x_words:
        train_X[i + len(train), :] += np.array(model[x_w])

In [11]:
test_X = np.zeros((len(test), len(model['hello'])))
for i in range(len(test)):
    test_words = test[i].split(' ')
    for x_w in test_words:
        test_X[i, :] += np.array(model[x_w])

In [12]:
train_y = np.zeros(len(train) + len(id_rubric), dtype='int')
for i in range(len(train)):
    train_y[i] = rubric_id[train[i][1]]
for i in range(len(id_rubric)):
    train_y[i + len(train)] = i

In [13]:
import xgboost
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_jobs = -1)
#clf = xgboost.XGBClassifier(n_jobs=-1)
#clf = RandomForestClassifier(n_estimators=100, max_depth = 5, verbose=2, n_jobs=-1)
#clf = AdaBoostClassifier(n_estimators=5)

In [14]:
del model
del test
del train

In [15]:
clf.fit(train_X, train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

In [16]:
pred = clf.predict(test_X)
out = open('prediction.tsv', 'w')
for i in pred:
    out.write(id_rubric[i] + '\n')
out.close()

In [17]:
del pred

In [18]:
train_pred = clf.predict(train_X)
from sklearn.metrics import accuracy_score
print('Accuracy on train set', accuracy_score(train_pred, train_y))

Accuracy on train set 0.5196797333767083
